In [1]:
import PyPDF2
import glob
import numpy as np
import pdfplumber
from collections import defaultdict
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure

In [2]:
def text_extraction(element):
    
    line_text = element.get_text()
    
    line_formats = []
    for text_line in element:
        
        if isinstance(text_line, LTTextContainer):
            
            for character in text_line:
                
                if isinstance(character, LTChar):
                    
                    line_formats.append(character.fontname)
                    
                    line_formats.append(character.size)
                    
    format_per_line = list(set(line_formats))
    
    return (line_text, format_per_line)

def analyst_executives(path):
    
    time=0
    presentation_index=0
    Elon_Musk_index=0
    Question_and_Answer_index=0
    Elon_Musk_speech=[]
    presentation_speech=[]
    text=[]
    executives_name=[]
    analysts_name=[]

    executives_title=[]
    analysts_title=[]

    executives_index=0
    analysts_index=0

    for pagenum, page in enumerate(extract_pages(path)):
        
        for _, element in enumerate(page):

            if pagenum == 2:

                if isinstance(element, LTTextContainer):
                    line_text, format_per_line = text_extraction(element)

                text.append(line_text)

                if line_text == "EXECUTIVES\n":
                    executives_index+=1

                if line_text == 'ANALYSTS\n':
                    executives_index=0

                if executives_index >= 1:
                    executives_name.append(line_text.split("\n")[0])
                    executives_title.append(line_text.split("\n")[1])

                if line_text == 'ANALYSTS\n':
                    analysts_index+=1
                    executives_index=0

                if 'S&P Global Market Intelligence' in line_text:
                    analysts_index=0

                if analysts_index >= 1:
                    analysts_name.append(line_text.split("\n")[0])    
                    analysts_title.append(line_text.split("\n")[1])

        if pagenum == 4:
            break

    ppl=defaultdict(lambda: defaultdict(str))

    for i in range(1, len(analysts_name), 1):

        ppl['analysts'][analysts_name[i]]=analysts_title[i]

    for i in range(1, len(executives_name), 1):

        ppl['executives'][executives_name[i]]=executives_title[i]
    
    return ppl

def processing(path):
    
    presentation_index=0
    Elon_Musk_index=0
    Question_and_Answer_index=0
    Elon_Musk_speech=[]
    presentation_speech=[]
    time=0

    for pagenum, page in enumerate(extract_pages(path)):

        test=[]

        for _, element in enumerate(page):

            if isinstance(element, LTTextContainer):

                line_text, format_per_line = text_extraction(element)

            test.append(line_text)

            if pagenum == 0:
                if _ == 0:
                    time=line_text
                    time=time[61:100]

            if line_text == 'Presentation\n':
                presentation_index = presentation_index + 1

            if line_text == 'Question and Answer\n':
                Question_and_Answer_index = Question_and_Answer_index + 1

            if Question_and_Answer_index == 2:
                break

            if presentation_index == 2:
                presentation_speech.append(line_text)
    
    return time, presentation_speech

In [3]:
files=glob.glob("/Volumes/4TB/ConferenceCall/Tesla/Earningcall/*.pdf")

In [4]:
path=files[0]
print(path)
pdf=open(path, 'rb')
pdfReader=PyPDF2.PdfReader(pdf)

/Volumes/4TB/ConferenceCall/Tesla/Earningcall/Tesla Motors, Inc., Q1 2011 Earnings Call, May 04, 2011.pdf


In [5]:
ppl=analyst_executives(path)

ppl['executives'].items()
executives_people=[elem[0] for elem in ppl['executives'].items()]
executives_title=[elem[1] for elem in ppl['executives'].items()]

ppl['analysts'].items()
analysts_people=[elem[0] for elem in ppl['analysts'].items()]
analysts_title=[elem[1] for elem in ppl['analysts'].items()]

In [6]:
ppl

defaultdict(<function __main__.analyst_executives.<locals>.<lambda>()>,
            {'analysts': defaultdict(str,
                         {'Adam Michael Jonas': 'Morgan Stanley, Research Division',
                          'Andrea Susan James': 'Dougherty & Company LLC,',
                          'Daniel V. Galves': 'Deutsche Bank AG, Research',
                          'Himanshu A. Patel': 'JP Morgan Chase & Co, Research',
                          'Patrick Kenehan Archambault': 'Goldman Sachs Group Inc.,',
                          'Unknown Analyst': ''}),
             'executives': defaultdict(str,
                         {'Deepak Ahuja': 'Former Chief Financial Officer',
                          'Elon R. Musk': 'Co-Founder, CEO, Director &',
                          'Jeffrey K. Evanson': 'Former Vice President of Investor'})})

In [7]:
time, presentation_speech=processing(path)

In [8]:
def speech_dict(presentation_speech, executives):

    speech_people=defaultdict(lambda: defaultdict(str))
    speech_index_dict=defaultdict(lambda: defaultdict(str))
    speech_index=0

    for _, text in enumerate(presentation_speech):
        text=text.split("\n")
        for executives in executives_people:
            if executives in text[0]:
                speech_people[str("Presentation_")+str(speech_index)]=executives
                speech_index_dict[str("Presentation_")+str(speech_index)]=_+1
                speech_index+=1

    # ================================================================================
    speech_speech_index_list=[]

    for i in speech_index_dict.items():

        speech_speech_index_list.append(i[1])

    speech_speech_index_list.append(len(presentation_speech))

    # ================================================================================
    length=len(speech_index_dict.items())
    speech_speech_index_all=defaultdict(lambda: defaultdict(list))

    for i in range(length):

        rang=range(speech_speech_index_list[i], speech_speech_index_list[i+1], 1)

        list_=[x for x in rang]

        speech_speech_index_all[str("Presentation_")+str(i)]=list_

    speech_speech_dict=defaultdict(lambda: defaultdict(list))
    speech_index=0

    for i in speech_speech_index_all.items():

        for j in i[1]:

            speech_speech_dict[i[0]][j].append(presentation_speech[j])
            
    return speech_speech_dict, speech_people

In [12]:
speech_speech_dict, speech_people = speech_dict(presentation_speech, executives_people)

In [13]:
speech_speech_dict

defaultdict(<function __main__.speech_dict.<locals>.<lambda>()>,
            {'Presentation_0': defaultdict(list,
                         {4: ["Thank you, Tyrone, and thank you all for joining us this afternoon. Welcome to Tesla Motors earnings call\nfor the first quarter of 2011. With me on the call today are Elon Musk, Chairman, Product Architect and\nCEO of Tesla Motors; and Deepak Ahuja, Tesla's Chief Financial Officer.\n"],
                          5: ["During the course of this conference call, we will discuss our business outlook and make other forward-\nlooking statements within the meaning of the safe harbor provisions of the Private Securities Litigation\nReform Act of 1995. Such statements are only predictions based on management's current expectations.\nActual events or results could differ materially from those predictions due to a number of risks and\nuncertainties, including those discussed in the Risk Factors section of our most recent form 10-K filed on\nMarch 3rd of

In [14]:
ppl

defaultdict(<function __main__.analyst_executives.<locals>.<lambda>()>,
            {'analysts': defaultdict(str,
                         {'Adam Michael Jonas': 'Morgan Stanley, Research Division',
                          'Andrea Susan James': 'Dougherty & Company LLC,',
                          'Daniel V. Galves': 'Deutsche Bank AG, Research',
                          'Himanshu A. Patel': 'JP Morgan Chase & Co, Research',
                          'Patrick Kenehan Archambault': 'Goldman Sachs Group Inc.,',
                          'Unknown Analyst': ''}),
             'executives': defaultdict(str,
                         {'Deepak Ahuja': 'Former Chief Financial Officer',
                          'Elon R. Musk': 'Co-Founder, CEO, Director &',
                          'Jeffrey K. Evanson': 'Former Vice President of Investor'})})

In [15]:
speech_people

defaultdict(<function __main__.speech_dict.<locals>.<lambda>()>,
            {'Presentation_0': 'Jeffrey K. Evanson',
             'Presentation_1': 'Elon R. Musk',
             'Presentation_2': 'Deepak Ahuja',
             'Presentation_3': 'Jeffrey K. Evanson'})

In [16]:
time

'Wednesday, May 04, 2011 9:00 PM GMT\nS&P'